In [1]:
import numpy as np
import warnings
from tqdm import tqdm
import shap
import pandas as pd
import joblib
import shap

X_test = pd.read_csv("/ml_data/X_test.csv").drop(columns = "Unnamed: 0")
X_test["built_year"] = X_test["built_year"].astype('category')
y_test = pd.read_csv("/ml_data/y_test.csv").drop(columns = "Unnamed: 0")
X_train_1 = pd.read_csv("/ml_data/transform.csv").drop(columns = ["Unnamed: 0"])
feature_names = list(X_test.columns)
model = joblib.load("/artifact/mlruns/286978358198217900/1e69c74027c745b6819ce40a8f6de458/artifacts/model/model.pkl")
preprocessor_1 = joblib.load("/save_pipeline/pipeline_09_01_23_02:13:43.pkl")
preprocessor_1_ = joblib.load("/save_pipeline/pipeline_09_01_23_02:13:43.pkl")
preprocessor_2 = preprocessor_1_.fit(X_test)

def check_cols(preprocessor):
    all_columns = []
    for i in range(len(preprocessor.transformers_)):
        if preprocessor.transformers_[i][0] == "num":
            all_columns.extend(preprocessor.transformers[i][2])
        else:
            for j in range(len(preprocessor.transformers_[1][1].steps)):
                for array_ja in preprocessor.transformers_[1][1].steps[j][1].categories_:
                    all_columns.extend(list(array_ja))
    return all_columns

def check_type(df)-> list[str]:
    df_columns = list(df.columns)
    numeric = []
    catagories = []
    for values in df.columns:
        if df[values].dtypes == int or df[values].dtypes == float or df[values].dtypes == 'uint8':
            numeric.append(values)
        else:
            catagories.append(values)
    return numeric

def insert_cols(prepro_1, prepro_2):
    num = []
    j = -1
    for i in check_cols(prepro_1):
        j += 1
        if i not in check_cols(prepro_2):
            num.append(j)
    return num


def save_to_csv():
    all_columns = check_cols(preprocessor_1)
    explainer = shap.TreeExplainer(model[0] ,feature_names = all_columns)
    count_num = len(check_type(X_test))
    for_pandas = preprocessor_1.fit_transform(X_test)
    X_test_ja = pd.DataFrame(np.insert(for_pandas, insert_cols(preprocessor_1, preprocessor_2), 0, axis = 1), columns = all_columns)
    X_test_ja.iloc[:,:count_num] = preprocessor_1.transformers_[0][1].steps[1][1].inverse_transform(X_test_ja.iloc[:,:count_num])
    shap_values = explainer.shap_values(np.insert(for_pandas, insert_cols(preprocessor_1, preprocessor_2), 0, axis = 1), y = y_test.values)
    global_values = pd.DataFrame(np.reshape(sum(np.abs(shap_values))/len(sum(np.abs(shap_values))), newshape = (1,-1)), columns = all_columns)
    train_dependency_values = pd.DataFrame(np.abs(shap_values), columns = ["importance_values_" + i for i in all_columns])
    pd.concat([train_dependency_values, X_test_ja], axis = 1).to_csv("/model_interpret/local.csv")
    global_values.to_csv("/model_interpret/global.csv")


In [10]:
X_train_1.iloc[:,96].unique()

array([0., 1.])

In [2]:
len(check_cols(preprocessor_2))

134

In [3]:
insert_cols(preprocessor_1, preprocessor_2)

[96, 97, 98, 126]

In [4]:
all_columns = check_cols(preprocessor_1)
add_cols = insert_cols(preprocessor_1, preprocessor_2)
explainer = shap.TreeExplainer(model[0] ,feature_names = all_columns)
count_num = len(check_type(X_test))
for_pandas = preprocessor_1.fit_transform(X_test)
X_test_ja = pd.DataFrame(np.insert(for_pandas, add_cols, 0, axis = 1), columns = all_columns)
X_test_ja.iloc[:,:count_num] = preprocessor_1.transformers_[0][1].steps[1][1].inverse_transform(X_test_ja.iloc[:,:count_num])
shap_values = explainer.shap_values(np.insert(for_pandas, add_cols, 0, axis = 1))
global_values = pd.DataFrame(np.reshape(sum(np.abs(shap_values))/len(sum(np.abs(shap_values))), newshape = (1,-1)), columns = all_columns)
train_dependency_values = pd.DataFrame(np.abs(shap_values), columns = ["importance_values_" + str(i) for i in all_columns])
# pd.concat([train_dependency_values, X_test_ja], axis = 1).to_csv("/model_interpret/local.csv")
# global_values.to_csv("/model_interpret/global.csv")

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


In [5]:
count_num

78

In [6]:
for_pandas.shape

(1274, 134)

In [7]:
print(type(insert_cols(preprocessor_1, preprocessor_2)))

<class 'list'>
